In [79]:
%matplotlib notebook

import matplotlib.pyplot as plt
from IPython.display import display
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import os
import pickle

#Change this for each participant
part_number = '201'

# Set matplotlib parameters for displaying graphs
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = [9, 4.5]  # Bigger images
plt.rcParams['font.size']= 14
pd.set_option('display.max_columns', None)

save_dir = '../data/part'+part_number+'/figures/'

p = '../data/part'+part_number+'/part'+part_number+'_pilot_ecg_signals.p'
ecg = pickle.load(open(p,"rb"))
eda = pickle.load(open('../data/part'+part_number+'/part'+part_number+'_pilot_eda_signals.p',"rb"))
rsp = pickle.load(open('../data/part'+part_number+'/part'+part_number+'_pilot_rsp_signals.p',"rb"))
baseline_fn = '../data/part'+part_number+'/part'+part_number+'_baseline_ecg_signals.p'
baseline2_fn = '../data/part'+part_number+'/part'+part_number+'_baseline_eda_signals.p'
baseline3_fn = '../data/part'+part_number+'/part'+part_number+'_baseline_rsp_signals.p'
ecg_baseline = pickle.load(open(baseline_fn,"rb"))
eda_baseline = pickle.load(open(baseline2_fn,"rb"))
rsp_baseline = pickle.load(open(baseline3_fn,"rb"))

data_file = '../data/part'+part_number+'/part'+part_number+'_pilot.acq'
rate = 2000

data, sampling_rate = nk.read_acqknowledge(data_file)
data = data.rename(columns={"RSP, X, RSPEC-R": "RSP", "DTU100 - Trigger View, AMI / HLT - A11": "TRIG",
                            "EDA, X, PPGED-R": "EDA", "ECG, X, RSPEC-R": "ECG"})

timestamps=np.loadtxt('../support/trigger timestamps/trigger timestamps.csv',dtype='int',delimiter=',')
start_times = timestamps[int(part_number[-2:]) - 1]
feedback_times = timestamps[int(part_number[-2:]) - 1 + 15]

taskloads_all = np.loadtxt('../support/taskload settings/taskload settings.csv',dtype='int',delimiter=',',encoding='UTF-8')
taskload_settings = taskloads_all[int(part_number[-2:]) - 1]

durations = [100,100,100,100,100,100,100,100,100,100,100,100]
#durations = [633339, 472624, 462436, 479671, 443789, 460186, 451913, 467455, 465081, 407503, 453908, 453551]

events = {}
events['onset'] = start_times
events['duration'] = durations
events['label']=  [1,2,3,4,5,6,7,8,9,10,11,12]
events['condition'] =  taskload_settings
#events['trust'] = trust_settings DOES NOT WORK!

events_feedback = {}
events_feedback['onset'] = feedback_times
events_feedback['duration'] = durations
events_feedback['label']=  [1,2,3,4,5,6,7,8,9,10,11,12]
events_feedback['condition'] =  taskload_settings

ecg_pilot_interval = nk.epochs_create(ecg, events, sampling_rate=rate, epochs_start=0, epochs_end=50)
eda_pilot_interval = nk.epochs_create(eda, events, sampling_rate=rate, epochs_start=0, epochs_end=50)
rsp_pilot_interval = nk.epochs_create(rsp, events, sampling_rate=rate, epochs_start=0, epochs_end=50)

ecg_trust_interval = nk.epochs_create(ecg,events,sampling_rate=rate,epochs_start=50,epochs_end=70)
eda_trust_interval = nk.epochs_create(eda,events,sampling_rate=rate,epochs_start=50,epochs_end=70)
rsp_trust_interval = nk.epochs_create(rsp,events,sampling_rate=rate,epochs_start=50,epochs_end=70)

ecg_trust_event = nk.epochs_create(ecg,events_feedback,sampling_rate=rate,epochs_start=-5,epochs_end=20)
eda_trust_event = nk.epochs_create(eda,events_feedback,sampling_rate=rate,epochs_start=-5,epochs_end=20)
rsp_trust_event = nk.epochs_create(rsp,events_feedback,sampling_rate=rate,epochs_start=-5,epochs_end=20)

pi_high_tl = {}
hc=1
pi_med_tl = {}
mc=1
pi_low_tl = {}
lc=1
for i in range(1,13):
    if (ecg_pilot_interval[i]['Condition'].iloc[0] == 4.0):
        pi_high_tl[hc] = ecg_pilot_interval[i]
        hc+=1
    if (ecg_pilot_interval[i]['Condition'].iloc[0] == 3.0):
        pi_med_tl[mc] = ecg_pilot_interval[i]
        mc+=1
    if (ecg_pilot_interval[i]['Condition'].iloc[0] == 2.0):
        pi_low_tl[lc] = ecg_pilot_interval[i]
        lc+=1

for i in range(1,13):
    if (eda_pilot_interval[i]['Condition'].iloc[0] == 4.0):
        pi_high_tl[hc] = eda_pilot_interval[i]
        hc+=1
    if (eda_pilot_interval[i]['Condition'].iloc[0] == 3.0):
        pi_med_tl[mc] = eda_pilot_interval[i]
        mc+=1
    if (eda_pilot_interval[i]['Condition'].iloc[0] == 2.0):
        pi_low_tl[lc] = eda_pilot_interval[i]
        lc+=1
            
for i in range(1,13):
    if (rsp_pilot_interval[i]['Condition'].iloc[0] == 4.0):
        pi_high_tl[hc] = rsp_pilot_interval[i]
        hc+=1
    if (rsp_pilot_interval[i]['Condition'].iloc[0] == 3.0):
        pi_med_tl[mc] = rsp_pilot_interval[i]
        mc+=1
    if (rsp_pilot_interval[i]['Condition'].iloc[0] == 2.0):
        pi_low_tl[lc] = rsp_pilot_interval[i]
        lc+=1
        
x_axis_pilot = np.linspace(0, ecg_pilot_interval[1].shape[0]/ sampling_rate, ecg_pilot_interval[1].shape[0])

In [91]:
[timestamps[int(part_number[-2:]) - 1 + 15]]

[array([ 950589, 1495874, 2029686, 2580921, 3103039, 3767436, 4295163,
        4844705, 5378331, 5858753, 6393158, 6956801])]

In [80]:
fig, ecg_pilot_axs = plt.subplots(4, 3,figsize=(20,10))
ecg_pilot_axs[0,0].plot(x_axis_pilot, ecg_pilot_interval[1]['ECG_Clean'])

ecg_pilot_axs[0,0].set_ylabel('Voltage')
ecg_pilot_axs[0,0].set_title('Trial 1')
#ecg_pilot_axs[0,0].set_ylim(-.5,1)

ecg_pilot_axs[0,1].plot(x_axis_pilot, ecg_pilot_interval[2]['ECG_Clean'])

ecg_pilot_axs[0,1].set_title('Trial 2')
#ecg_pilot_axs[0,1].set_ylim(-.5,1)

ecg_pilot_axs[0,2].plot(x_axis_pilot, ecg_pilot_interval[3]['ECG_Clean'])

ecg_pilot_axs[0,2].set_title('Trial 3')
#ecg_pilot_axs[0,2].set_ylim(-.5,1)

ecg_pilot_axs[1,0].plot(x_axis_pilot, ecg_pilot_interval[4]['ECG_Clean'])

ecg_pilot_axs[1,0].set_ylabel('Voltage')
ecg_pilot_axs[1,0].set_title('Trial 4')
#ecg_pilot_axs[1,0].set_ylim(-.5,1)

ecg_pilot_axs[1,1].plot(x_axis_pilot, ecg_pilot_interval[5]['ECG_Clean'])

ecg_pilot_axs[1,1].set_title('Trial 5')
#ecg_pilot_axs[1,1].set_ylim(-.5,1)

ecg_pilot_axs[1,2].plot(x_axis_pilot, ecg_pilot_interval[6]['ECG_Clean'])

ecg_pilot_axs[1,2].set_title('Trial 6')
#ecg_pilot_axs[1,2].set_ylim(-.5,1)

ecg_pilot_axs[2,0].plot(x_axis_pilot, ecg_pilot_interval[7]['ECG_Clean'])

ecg_pilot_axs[2,0].set_ylabel('Voltage')
ecg_pilot_axs[2,0].set_title('Trial 7')
#ecg_pilot_axs[2,0].set_ylim(-.5,1)

ecg_pilot_axs[2,1].plot(x_axis_pilot, ecg_pilot_interval[8]['ECG_Clean'])

ecg_pilot_axs[2,1].set_title('Trial 8')
#ecg_pilot_axs[2,1].set_ylim(-.5,1)

ecg_pilot_axs[2,2].plot(x_axis_pilot, ecg_pilot_interval[9]['ECG_Clean'])

ecg_pilot_axs[2,2].set_title('Trial 9')
#ecg_pilot_axs[2,2].set_ylim(-.5,1)

ecg_pilot_axs[3,0].plot(x_axis_pilot, ecg_pilot_interval[10]['ECG_Clean'])
ecg_pilot_axs[3,0].set_xlabel('Sec')
ecg_pilot_axs[3,0].set_ylabel('Voltage')
ecg_pilot_axs[3,0].set_title('Trial 10')
#ecg_pilot_axs[3,0].set_ylim(-.5,1)

ecg_pilot_axs[3,1].plot(x_axis_pilot, ecg_pilot_interval[11]['ECG_Clean'])
ecg_pilot_axs[3,1].set_xlabel('Sec')

ecg_pilot_axs[3,1].set_title('Trial 11')
#ecg_pilot_axs[3,1].set_ylim(-.5,1)

ecg_pilot_axs[3,2].plot(x_axis_pilot, ecg_pilot_interval[12]['ECG_Clean'])
ecg_pilot_axs[3,2].set_xlabel('Sec')
ecg_pilot_axs[3,2].set_title('Trial 12')
#ecg_pilot_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' ECG Piloting Interval Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

In [81]:
fig, eda_pilot_axs = plt.subplots(4, 3,figsize=(20,10))
eda_pilot_axs[0,0].plot(x_axis_pilot, eda_pilot_interval[1]['EDA_Clean'])

eda_pilot_axs[0,0].set_ylabel('μS')
eda_pilot_axs[0,0].set_title('Trial 1')
#eda_pilot_axs[0,0].set_ylim(-.5,1)

eda_pilot_axs[0,1].plot(x_axis_pilot, eda_pilot_interval[2]['EDA_Clean'])

eda_pilot_axs[0,1].set_title('Trial 2')
#eda_pilot_axs[0,1].set_ylim(-.5,1)

eda_pilot_axs[0,2].plot(x_axis_pilot, eda_pilot_interval[3]['EDA_Clean'])

eda_pilot_axs[0,2].set_title('Trial 3')
#eda_pilot_axs[0,2].set_ylim(-.5,1)

eda_pilot_axs[1,0].plot(x_axis_pilot, eda_pilot_interval[4]['EDA_Clean'])

eda_pilot_axs[1,0].set_ylabel('μS')
eda_pilot_axs[1,0].set_title('Trial 4')
#eda_pilot_axs[1,0].set_ylim(-.5,1)

eda_pilot_axs[1,1].plot(x_axis_pilot, eda_pilot_interval[5]['EDA_Clean'])

eda_pilot_axs[1,1].set_title('Trial 5')
#eda_pilot_axs[1,1].set_ylim(-.5,1)

eda_pilot_axs[1,2].plot(x_axis_pilot, eda_pilot_interval[6]['EDA_Clean'])

eda_pilot_axs[1,2].set_title('Trial 6')
#eda_pilot_axs[1,2].set_ylim(-.5,1)

eda_pilot_axs[2,0].plot(x_axis_pilot, eda_pilot_interval[7]['EDA_Clean'])

eda_pilot_axs[2,0].set_ylabel('μS')
eda_pilot_axs[2,0].set_title('Trial 7')
#eda_pilot_axs[2,0].set_ylim(-.5,1)

eda_pilot_axs[2,1].plot(x_axis_pilot, eda_pilot_interval[8]['EDA_Clean'])

eda_pilot_axs[2,1].set_title('Trial 8')
#eda_pilot_axs[2,1].set_ylim(-.5,1)

eda_pilot_axs[2,2].plot(x_axis_pilot, eda_pilot_interval[9]['EDA_Clean'])

eda_pilot_axs[2,2].set_title('Trial 9')
#eda_pilot_axs[2,2].set_ylim(-.5,1)

eda_pilot_axs[3,0].plot(x_axis_pilot, eda_pilot_interval[10]['EDA_Clean'])
eda_pilot_axs[3,0].set_xlabel('Sec')
eda_pilot_axs[3,0].set_ylabel('μS')
eda_pilot_axs[3,0].set_title('Trial 10')
#eda_pilot_axs[3,0].set_ylim(-.5,1)

eda_pilot_axs[3,1].plot(x_axis_pilot, eda_pilot_interval[11]['EDA_Clean'])
eda_pilot_axs[3,1].set_xlabel('Sec')

eda_pilot_axs[3,1].set_title('Trial 11')
#eda_pilot_axs[3,1].set_ylim(-.5,1)

eda_pilot_axs[3,2].plot(x_axis_pilot, eda_pilot_interval[12]['EDA_Clean'])
eda_pilot_axs[3,2].set_xlabel('Sec')
eda_pilot_axs[3,2].set_title('Trial 12')
#eda_pilot_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' EDA Piloting Interval Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

In [82]:
fig, rsp_pilot_axs = plt.subplots(4, 3,figsize=(20,10))
rsp_pilot_axs[0,0].plot(x_axis_pilot, rsp_pilot_interval[1]['RSP_Clean'])

rsp_pilot_axs[0,0].set_ylabel('Voltage')
rsp_pilot_axs[0,0].set_title('Trial 1')
#rsp_pilot_axs[0,0].set_ylim(-.5,1)

rsp_pilot_axs[0,1].plot(x_axis_pilot, rsp_pilot_interval[2]['RSP_Clean'])

rsp_pilot_axs[0,1].set_title('Trial 2')
#rsp_pilot_axs[0,1].set_ylim(-.5,1)

rsp_pilot_axs[0,2].plot(x_axis_pilot, rsp_pilot_interval[3]['RSP_Clean'])

rsp_pilot_axs[0,2].set_title('Trial 3')
#rsp_pilot_axs[0,2].set_ylim(-.5,1)

rsp_pilot_axs[1,0].plot(x_axis_pilot, rsp_pilot_interval[4]['RSP_Clean'])

rsp_pilot_axs[1,0].set_ylabel('Voltage')
rsp_pilot_axs[1,0].set_title('Trial 4')
#rsp_pilot_axs[1,0].set_ylim(-.5,1)

rsp_pilot_axs[1,1].plot(x_axis_pilot, rsp_pilot_interval[5]['RSP_Clean'])

rsp_pilot_axs[1,1].set_title('Trial 5')
#rsp_pilot_axs[1,1].set_ylim(-.5,1)

rsp_pilot_axs[1,2].plot(x_axis_pilot, rsp_pilot_interval[6]['RSP_Clean'])

rsp_pilot_axs[1,2].set_title('Trial 6')
#rsp_pilot_axs[1,2].set_ylim(-.5,1)

rsp_pilot_axs[2,0].plot(x_axis_pilot, rsp_pilot_interval[7]['RSP_Clean'])

rsp_pilot_axs[2,0].set_ylabel('Voltage')
rsp_pilot_axs[2,0].set_title('Trial 7')
#rsp_pilot_axs[2,0].set_ylim(-.5,1)

rsp_pilot_axs[2,1].plot(x_axis_pilot, rsp_pilot_interval[8]['RSP_Clean'])

rsp_pilot_axs[2,1].set_title('Trial 8')
#rsp_pilot_axs[2,1].set_ylim(-.5,1)

rsp_pilot_axs[2,2].plot(x_axis_pilot, rsp_pilot_interval[9]['RSP_Clean'])

rsp_pilot_axs[2,2].set_title('Trial 9')
#rsp_pilot_axs[2,2].set_ylim(-.5,1)

rsp_pilot_axs[3,0].plot(x_axis_pilot, rsp_pilot_interval[10]['RSP_Clean'])
rsp_pilot_axs[3,0].set_xlabel('Sec')
rsp_pilot_axs[3,0].set_ylabel('Voltage')
rsp_pilot_axs[3,0].set_title('Trial 10')
#rsp_pilot_axs[3,0].set_ylim(-.5,1)

rsp_pilot_axs[3,1].plot(x_axis_pilot, rsp_pilot_interval[11]['RSP_Clean'])
rsp_pilot_axs[3,1].set_xlabel('Sec')

rsp_pilot_axs[3,1].set_title('Trial 11')
#rsp_pilot_axs[3,1].set_ylim(-.5,1)

rsp_pilot_axs[3,2].plot(x_axis_pilot, rsp_pilot_interval[12]['RSP_Clean'])
rsp_pilot_axs[3,2].set_xlabel('Sec')
rsp_pilot_axs[3,2].set_title('Trial 12')
#rsp_pilot_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' RSP Piloting Interval Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

In [83]:
x_axis_trust = np.linspace(0, ecg_trust_interval[1].shape[0]/ sampling_rate, ecg_trust_interval[1].shape[0])

In [84]:
fig, ecg_trust_axs = plt.subplots(4, 3,figsize=(20,10))
ecg_trust_axs[0,0].plot(x_axis_trust, ecg_trust_interval[1]['ECG_Clean'])

ecg_trust_axs[0,0].set_ylabel('Voltage')
ecg_trust_axs[0,0].set_title('Trial 1')
#ecg_trust_axs[0,0].set_ylim(-.5,1)

ecg_trust_axs[0,1].plot(x_axis_trust, ecg_trust_interval[2]['ECG_Clean'])

ecg_trust_axs[0,1].set_title('Trial 2')
#ecg_trust_axs[0,1].set_ylim(-.5,1)

ecg_trust_axs[0,2].plot(x_axis_trust, ecg_trust_interval[3]['ECG_Clean'])

ecg_trust_axs[0,2].set_title('Trial 3')
#ecg_trust_axs[0,2].set_ylim(-.5,1)

ecg_trust_axs[1,0].plot(x_axis_trust, ecg_trust_interval[4]['ECG_Clean'])

ecg_trust_axs[1,0].set_ylabel('Voltage')
ecg_trust_axs[1,0].set_title('Trial 4')
#ecg_trust_axs[1,0].set_ylim(-.5,1)

ecg_trust_axs[1,1].plot(x_axis_trust, ecg_trust_interval[5]['ECG_Clean'])

ecg_trust_axs[1,1].set_title('Trial 5')
#ecg_trust_axs[1,1].set_ylim(-.5,1)

ecg_trust_axs[1,2].plot(x_axis_trust, ecg_trust_interval[6]['ECG_Clean'])

ecg_trust_axs[1,2].set_title('Trial 6')
#ecg_trust_axs[1,2].set_ylim(-.5,1)

ecg_trust_axs[2,0].plot(x_axis_trust, ecg_trust_interval[7]['ECG_Clean'])

ecg_trust_axs[2,0].set_ylabel('Voltage')
ecg_trust_axs[2,0].set_title('Trial 7')
#ecg_trust_axs[2,0].set_ylim(-.5,1)

ecg_trust_axs[2,1].plot(x_axis_trust, ecg_trust_interval[8]['ECG_Clean'])

ecg_trust_axs[2,1].set_title('Trial 8')
#ecg_trust_axs[2,1].set_ylim(-.5,1)

ecg_trust_axs[2,2].plot(x_axis_trust, ecg_trust_interval[9]['ECG_Clean'])

ecg_trust_axs[2,2].set_title('Trial 9')
#ecg_trust_axs[2,2].set_ylim(-.5,1)

ecg_trust_axs[3,0].plot(x_axis_trust, ecg_trust_interval[10]['ECG_Clean'])
ecg_trust_axs[3,0].set_xlabel('Sec')
ecg_trust_axs[3,0].set_ylabel('Voltage')
ecg_trust_axs[3,0].set_title('Trial 10')
#ecg_trust_axs[3,0].set_ylim(-.5,1)

ecg_trust_axs[3,1].plot(x_axis_trust, ecg_trust_interval[11]['ECG_Clean'])
ecg_trust_axs[3,1].set_xlabel('Sec')

ecg_trust_axs[3,1].set_title('Trial 11')
#ecg_trust_axs[3,1].set_ylim(-.5,1)

ecg_trust_axs[3,2].plot(x_axis_trust, ecg_trust_interval[12]['ECG_Clean'])
ecg_trust_axs[3,2].set_xlabel('Sec')
ecg_trust_axs[3,2].set_title('Trial 12')
#ecg_trust_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' ECG Trust Interval Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

In [85]:
fig, eda_trust_axs = plt.subplots(4, 3,figsize=(20,10))
eda_trust_axs[0,0].plot(x_axis_trust, eda_trust_interval[1]['EDA_Clean'])

eda_trust_axs[0,0].set_ylabel('μS')
eda_trust_axs[0,0].set_title('Trial 1')
#eda_trust_axs[0,0].set_ylim(-.5,1)

eda_trust_axs[0,1].plot(x_axis_trust, eda_trust_interval[2]['EDA_Clean'])

eda_trust_axs[0,1].set_title('Trial 2')
#eda_trust_axs[0,1].set_ylim(-.5,1)

eda_trust_axs[0,2].plot(x_axis_trust, eda_trust_interval[3]['EDA_Clean'])

eda_trust_axs[0,2].set_title('Trial 3')
#eda_trust_axs[0,2].set_ylim(-.5,1)

eda_trust_axs[1,0].plot(x_axis_trust, eda_trust_interval[4]['EDA_Clean'])

eda_trust_axs[1,0].set_ylabel('μS')
eda_trust_axs[1,0].set_title('Trial 4')
#eda_trust_axs[1,0].set_ylim(-.5,1)

eda_trust_axs[1,1].plot(x_axis_trust, eda_trust_interval[5]['EDA_Clean'])

eda_trust_axs[1,1].set_title('Trial 5')
#eda_trust_axs[1,1].set_ylim(-.5,1)

eda_trust_axs[1,2].plot(x_axis_trust, eda_trust_interval[6]['EDA_Clean'])

eda_trust_axs[1,2].set_title('Trial 6')
#eda_trust_axs[1,2].set_ylim(-.5,1)

eda_trust_axs[2,0].plot(x_axis_trust, eda_trust_interval[7]['EDA_Clean'])

eda_trust_axs[2,0].set_ylabel('μS')
eda_trust_axs[2,0].set_title('Trial 7')
#eda_trust_axs[2,0].set_ylim(-.5,1)

eda_trust_axs[2,1].plot(x_axis_trust, eda_trust_interval[8]['EDA_Clean'])

eda_trust_axs[2,1].set_title('Trial 8')
#eda_trust_axs[2,1].set_ylim(-.5,1)

eda_trust_axs[2,2].plot(x_axis_trust, eda_trust_interval[9]['EDA_Clean'])

eda_trust_axs[2,2].set_title('Trial 9')
#eda_trust_axs[2,2].set_ylim(-.5,1)

eda_trust_axs[3,0].plot(x_axis_trust, eda_trust_interval[10]['EDA_Clean'])
eda_trust_axs[3,0].set_xlabel('Sec')
eda_trust_axs[3,0].set_ylabel('μS')
eda_trust_axs[3,0].set_title('Trial 10')
#eda_trust_axs[3,0].set_ylim(-.5,1)

eda_trust_axs[3,1].plot(x_axis_trust, eda_trust_interval[11]['EDA_Clean'])
eda_trust_axs[3,1].set_xlabel('Sec')

eda_trust_axs[3,1].set_title('Trial 11')
#eda_trust_axs[3,1].set_ylim(-.5,1)

eda_trust_axs[3,2].plot(x_axis_trust, eda_trust_interval[12]['EDA_Clean'])
eda_trust_axs[3,2].set_xlabel('Sec')
eda_trust_axs[3,2].set_title('Trial 12')
#eda_trust_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' EDA Trust Interval Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

In [86]:
fig, rsp_trust_axs = plt.subplots(4, 3,figsize=(20,10))
rsp_trust_axs[0,0].plot(x_axis_trust, rsp_trust_interval[1]['RSP_Clean'])

rsp_trust_axs[0,0].set_ylabel('Voltage')
rsp_trust_axs[0,0].set_title('Trial 1')
#rsp_trust_axs[0,0].set_ylim(-.5,1)

rsp_trust_axs[0,1].plot(x_axis_trust, rsp_trust_interval[2]['RSP_Clean'])

rsp_trust_axs[0,1].set_title('Trial 2')
#rsp_trust_axs[0,1].set_ylim(-.5,1)

rsp_trust_axs[0,2].plot(x_axis_trust, rsp_trust_interval[3]['RSP_Clean'])

rsp_trust_axs[0,2].set_title('Trial 3')
#rsp_trust_axs[0,2].set_ylim(-.5,1)

rsp_trust_axs[1,0].plot(x_axis_trust, rsp_trust_interval[4]['RSP_Clean'])

rsp_trust_axs[1,0].set_ylabel('Voltage')
rsp_trust_axs[1,0].set_title('Trial 4')
#rsp_trust_axs[1,0].set_ylim(-.5,1)

rsp_trust_axs[1,1].plot(x_axis_trust, rsp_trust_interval[5]['RSP_Clean'])

rsp_trust_axs[1,1].set_title('Trial 5')
#rsp_trust_axs[1,1].set_ylim(-.5,1)

rsp_trust_axs[1,2].plot(x_axis_trust, rsp_trust_interval[6]['RSP_Clean'])

rsp_trust_axs[1,2].set_title('Trial 6')
#rsp_trust_axs[1,2].set_ylim(-.5,1)

rsp_trust_axs[2,0].plot(x_axis_trust, rsp_trust_interval[7]['RSP_Clean'])

rsp_trust_axs[2,0].set_ylabel('Voltage')
rsp_trust_axs[2,0].set_title('Trial 7')
#rsp_trust_axs[2,0].set_ylim(-.5,1)

rsp_trust_axs[2,1].plot(x_axis_trust, rsp_trust_interval[8]['RSP_Clean'])

rsp_trust_axs[2,1].set_title('Trial 8')
#rsp_trust_axs[2,1].set_ylim(-.5,1)

rsp_trust_axs[2,2].plot(x_axis_trust, rsp_trust_interval[9]['RSP_Clean'])

rsp_trust_axs[2,2].set_title('Trial 9')
#rsp_trust_axs[2,2].set_ylim(-.5,1)

rsp_trust_axs[3,0].plot(x_axis_trust, rsp_trust_interval[10]['RSP_Clean'])
rsp_trust_axs[3,0].set_xlabel('Sec')
rsp_trust_axs[3,0].set_ylabel('Voltage')
rsp_trust_axs[3,0].set_title('Trial 10')
#rsp_trust_axs[3,0].set_ylim(-.5,1)

rsp_trust_axs[3,1].plot(x_axis_trust, rsp_trust_interval[11]['RSP_Clean'])
rsp_trust_axs[3,1].set_xlabel('Sec')

rsp_trust_axs[3,1].set_title('Trial 11')
#rsp_trust_axs[3,1].set_ylim(-.5,1)

rsp_trust_axs[3,2].plot(x_axis_trust, rsp_trust_interval[12]['RSP_Clean'])
rsp_trust_axs[3,2].set_xlabel('Sec')
rsp_trust_axs[3,2].set_title('Trial 12')
#rsp_trust_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' RSP Trust Interval Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

In [87]:
x_axis_trust_event = np.linspace(-5, ecg_trust_event[1].shape[0]/ sampling_rate - 5, ecg_trust_event[1].shape[0])

In [88]:
fig, ecg_trust_axs = plt.subplots(4, 3,figsize=(20,10))
ecg_trust_axs[0,0].plot(x_axis_trust_event, ecg_trust_event[1]['ECG_Clean'])

ecg_trust_axs[0,0].set_ylabel('Voltage')
ecg_trust_axs[0,0].set_title('Trial 1')
#ecg_trust_axs[0,0].set_ylim(-.5,1)

ecg_trust_axs[0,1].plot(x_axis_trust_event, ecg_trust_event[2]['ECG_Clean'])

ecg_trust_axs[0,1].set_title('Trial 2')
#ecg_trust_axs[0,1].set_ylim(-.5,1)

ecg_trust_axs[0,2].plot(x_axis_trust_event, ecg_trust_event[3]['ECG_Clean'])

ecg_trust_axs[0,2].set_title('Trial 3')
#ecg_trust_axs[0,2].set_ylim(-.5,1)

ecg_trust_axs[1,0].plot(x_axis_trust_event, ecg_trust_event[4]['ECG_Clean'])

ecg_trust_axs[1,0].set_ylabel('Voltage')
ecg_trust_axs[1,0].set_title('Trial 4')
#ecg_trust_axs[1,0].set_ylim(-.5,1)

ecg_trust_axs[1,1].plot(x_axis_trust_event, ecg_trust_event[5]['ECG_Clean'])

ecg_trust_axs[1,1].set_title('Trial 5')
#ecg_trust_axs[1,1].set_ylim(-.5,1)

ecg_trust_axs[1,2].plot(x_axis_trust_event, ecg_trust_event[6]['ECG_Clean'])

ecg_trust_axs[1,2].set_title('Trial 6')
#ecg_trust_axs[1,2].set_ylim(-.5,1)

ecg_trust_axs[2,0].plot(x_axis_trust_event, ecg_trust_event[7]['ECG_Clean'])

ecg_trust_axs[2,0].set_ylabel('Voltage')
ecg_trust_axs[2,0].set_title('Trial 7')
#ecg_trust_axs[2,0].set_ylim(-.5,1)

ecg_trust_axs[2,1].plot(x_axis_trust_event, ecg_trust_event[8]['ECG_Clean'])

ecg_trust_axs[2,1].set_title('Trial 8')
#ecg_trust_axs[2,1].set_ylim(-.5,1)

ecg_trust_axs[2,2].plot(x_axis_trust_event, ecg_trust_event[9]['ECG_Clean'])

ecg_trust_axs[2,2].set_title('Trial 9')
#ecg_trust_axs[2,2].set_ylim(-.5,1)

ecg_trust_axs[3,0].plot(x_axis_trust_event, ecg_trust_event[10]['ECG_Clean'])
ecg_trust_axs[3,0].set_xlabel('Sec')
ecg_trust_axs[3,0].set_ylabel('Voltage')
ecg_trust_axs[3,0].set_title('Trial 10')
#ecg_trust_axs[3,0].set_ylim(-.5,1)

ecg_trust_axs[3,1].plot(x_axis_trust_event, ecg_trust_event[11]['ECG_Clean'])
ecg_trust_axs[3,1].set_xlabel('Sec')

ecg_trust_axs[3,1].set_title('Trial 11')
#ecg_trust_axs[3,1].set_ylim(-.5,1)

ecg_trust_axs[3,2].plot(x_axis_trust_event, ecg_trust_event[12]['ECG_Clean'])
ecg_trust_axs[3,2].set_xlabel('Sec')
ecg_trust_axs[3,2].set_title('Trial 12')
#ecg_trust_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' ECG Trust Event Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


<IPython.core.display.Javascript object>

ValueError: x and y must have same first dimension, but have shapes (50000,) and (0,)

In [ ]:
ecg_trust_event

In [ ]:
fig, eda_trust_axs = plt.subplots(4, 3,figsize=(20,10))
eda_trust_axs[0,0].plot(x_axis_trust_event, eda_trust_event[1]['EDA_Clean'])

eda_trust_axs[0,0].set_ylabel('μS')
eda_trust_axs[0,0].set_title('Trial 1')
#eda_trust_axs[0,0].set_ylim(-.5,1)

eda_trust_axs[0,1].plot(x_axis_trust_event, eda_trust_event[2]['EDA_Clean'])

eda_trust_axs[0,1].set_title('Trial 2')
#eda_trust_axs[0,1].set_ylim(-.5,1)

eda_trust_axs[0,2].plot(x_axis_trust_event, eda_trust_event[3]['EDA_Clean'])

eda_trust_axs[0,2].set_title('Trial 3')
#eda_trust_axs[0,2].set_ylim(-.5,1)

eda_trust_axs[1,0].plot(x_axis_trust_event, eda_trust_event[4]['EDA_Clean'])

eda_trust_axs[1,0].set_ylabel('μS')
eda_trust_axs[1,0].set_title('Trial 4')
#eda_trust_axs[1,0].set_ylim(-.5,1)

eda_trust_axs[1,1].plot(x_axis_trust_event, eda_trust_event[5]['EDA_Clean'])

eda_trust_axs[1,1].set_title('Trial 5')
#eda_trust_axs[1,1].set_ylim(-.5,1)

eda_trust_axs[1,2].plot(x_axis_trust_event, eda_trust_event[6]['EDA_Clean'])

eda_trust_axs[1,2].set_title('Trial 6')
#eda_trust_axs[1,2].set_ylim(-.5,1)

eda_trust_axs[2,0].plot(x_axis_trust_event, eda_trust_event[7]['EDA_Clean'])

eda_trust_axs[2,0].set_ylabel('μS')
eda_trust_axs[2,0].set_title('Trial 7')
#eda_trust_axs[2,0].set_ylim(-.5,1)

eda_trust_axs[2,1].plot(x_axis_trust_event, eda_trust_event[8]['EDA_Clean'])

eda_trust_axs[2,1].set_title('Trial 8')
#eda_trust_axs[2,1].set_ylim(-.5,1)

eda_trust_axs[2,2].plot(x_axis_trust_event, eda_trust_event[9]['EDA_Clean'])

eda_trust_axs[2,2].set_title('Trial 9')
#eda_trust_axs[2,2].set_ylim(-.5,1)

eda_trust_axs[3,0].plot(x_axis_trust_event, eda_trust_event[10]['EDA_Clean'])
eda_trust_axs[3,0].set_xlabel('Sec')
eda_trust_axs[3,0].set_ylabel('μS')
eda_trust_axs[3,0].set_title('Trial 10')
#eda_trust_axs[3,0].set_ylim(-.5,1)

eda_trust_axs[3,1].plot(x_axis_trust_event, eda_trust_event[11]['EDA_Clean'])
eda_trust_axs[3,1].set_xlabel('Sec')

eda_trust_axs[3,1].set_title('Trial 11')
#eda_trust_axs[3,1].set_ylim(-.5,1)

eda_trust_axs[3,2].plot(x_axis_trust_event, eda_trust_event[12]['EDA_Clean'])
eda_trust_axs[3,2].set_xlabel('Sec')
eda_trust_axs[3,2].set_title('Trial 12')
#eda_trust_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' EDA Trust Event Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)


In [ ]:
fig, rsp_trust_axs = plt.subplots(4, 3,figsize=(20,10))
rsp_trust_axs[0,0].plot(x_axis_trust_event, rsp_trust_event[1]['RSP_Clean'])

rsp_trust_axs[0,0].set_ylabel('Voltage')
rsp_trust_axs[0,0].set_title('Trial 1')
#rsp_trust_axs[0,0].set_ylim(-.5,1)

rsp_trust_axs[0,1].plot(x_axis_trust_event, rsp_trust_event[2]['RSP_Clean'])

rsp_trust_axs[0,1].set_title('Trial 2')
#rsp_trust_axs[0,1].set_ylim(-.5,1)

rsp_trust_axs[0,2].plot(x_axis_trust_event, rsp_trust_event[3]['RSP_Clean'])

rsp_trust_axs[0,2].set_title('Trial 3')
#rsp_trust_axs[0,2].set_ylim(-.5,1)

rsp_trust_axs[1,0].plot(x_axis_trust_event, rsp_trust_event[4]['RSP_Clean'])

rsp_trust_axs[1,0].set_ylabel('Voltage')
rsp_trust_axs[1,0].set_title('Trial 4')
#rsp_trust_axs[1,0].set_ylim(-.5,1)

rsp_trust_axs[1,1].plot(x_axis_trust_event, rsp_trust_event[5]['RSP_Clean'])

rsp_trust_axs[1,1].set_title('Trial 5')
#rsp_trust_axs[1,1].set_ylim(-.5,1)

rsp_trust_axs[1,2].plot(x_axis_trust_event, rsp_trust_event[6]['RSP_Clean'])

rsp_trust_axs[1,2].set_title('Trial 6')
#rsp_trust_axs[1,2].set_ylim(-.5,1)

rsp_trust_axs[2,0].plot(x_axis_trust_event, rsp_trust_event[7]['RSP_Clean'])

rsp_trust_axs[2,0].set_ylabel('Voltage')
rsp_trust_axs[2,0].set_title('Trial 7')
#rsp_trust_axs[2,0].set_ylim(-.5,1)

rsp_trust_axs[2,1].plot(x_axis_trust_event, rsp_trust_event[8]['RSP_Clean'])

rsp_trust_axs[2,1].set_title('Trial 8')
#rsp_trust_axs[2,1].set_ylim(-.5,1)

rsp_trust_axs[2,2].plot(x_axis_trust_event, rsp_trust_event[9]['RSP_Clean'])

rsp_trust_axs[2,2].set_title('Trial 9')
#rsp_trust_axs[2,2].set_ylim(-.5,1)

rsp_trust_axs[3,0].plot(x_axis_trust_event, rsp_trust_event[10]['RSP_Clean'])
rsp_trust_axs[3,0].set_xlabel('Sec')
rsp_trust_axs[3,0].set_ylabel('Voltage')
rsp_trust_axs[3,0].set_title('Trial 10')
#rsp_trust_axs[3,0].set_ylim(-.5,1)

rsp_trust_axs[3,1].plot(x_axis_trust_event, rsp_trust_event[11]['RSP_Clean'])
rsp_trust_axs[3,1].set_xlabel('Sec')

rsp_trust_axs[3,1].set_title('Trial 11')
#rsp_trust_axs[3,1].set_ylim(-.5,1)

rsp_trust_axs[3,2].plot(x_axis_trust_event, rsp_trust_event[12]['RSP_Clean'])
rsp_trust_axs[3,2].set_xlabel('Sec')
rsp_trust_axs[3,2].set_title('Trial 12')
#rsp_trust_axs[3,2].set_ylim(-.5,1)

fig.tight_layout(pad=0)
title=part_number+' RSP Trust Event Trial Breakdown'
fig.suptitle(title)
plt.show()
plt.savefig(save_dir+title)
